# Lab-1.7: DPO 訓練

**目標**: 實現並訓練 DPO 對齊模型

In [ ]:
import torch
import torch.nn.functional as F

def dpo_loss(policy_logps_chosen, policy_logps_rejected, ref_logps_chosen, ref_logps_rejected, beta=0.1):
    chosen_rewards = beta * (policy_logps_chosen - ref_logps_chosen)
    rejected_rewards = beta * (policy_logps_rejected - ref_logps_rejected)
    loss = -F.logsigmoid(chosen_rewards - rejected_rewards).mean()
    return loss

print('✅ DPO 損失函數實現完成')